In [1]:
import json
from datetime import datetime

import pandas as pd
import requests
from sqlalchemy import select
from sqlalchemy.orm import aliased

from database import Database
from utils import DistributorApi

In [2]:
api = DistributorApi()
db = Database()

In [8]:
rules = api.get_stopsale_rules()
df_rules = api.stopsale_rules_to_df(rules)

Rules retrieved successfully.


In [9]:
df_rules.head()

,refundable,rate,max_release,dynamic_commission,age,edit_state,obsolete,name,description,updated_by,...,booking_date_from,booking_date_to,days_of_week_level,days_of_week_list,room_level,room_list,num_of_nights_level,num_of_nights_list,hours_level,hours_list
0,0,0,0,False,0,1,False,Cerrado todo menos retail - Hotel Regente / Av...,CDH-3221\nCDH-3624\n CDH-3222\n+ Huizhi\nCDH-2...,jesus.aenlle@traveltino.com,...,None,None,0,None,0,None,0,None,0,None
1,0,0,0,False,0,1,False,Blocked Provider for Hotel 113841,Automatic providers blocked algorithm.,AzureFunctions,...,None,None,0,None,0,None,0,None,0,None
2,0,0,0,False,0,1,False,Falkensteiner Resort Capo Boi,Requested by Mario 07/02/2020,pau.cespedes@logitravelgroup.com,...,None,None,0,None,0,None,0,None,0,[]
3,0,0,0,False,0,1,False,Stop Sale Hesperia DOTW,CDH-31979,aday.herreraplasencia@grupoveci.es,...,2024-12-20,2025-02-20,0,None,0,None,0,None,0,None
4,0,0,0,False,0,1,False,Blocked Provider for Hotel 1871350,Automatic providers blocked algorithm.,AzureFunctions,...,None,None,0,None,0,None,0,None,0,None


In [10]:
# clients_credential from discovery

cc = aliased(db.clients_credential)
df_credentials = pd.read_sql(
    sql=select(cc.c.id.label("credential_id"), cc.c.enabled), con=db.engine.connect()
)

In [11]:
# fitler for tags empty, product, customer request and credential list

df_rules_filtered = df_rules[
    (df_rules["credential_level"] == 1) & (df_rules["tag"].isin([-1, 1, 3]))
]
df_rules_expanded = df_rules_filtered.explode("credential_list").reset_index(drop=True)

In [12]:
df_rules_filtered.shape

(1200, 44)

In [13]:
df_rules_expanded["credential_id"] = df_rules_expanded["credential_list"]
df_rules_expanded = df_rules_expanded.merge(
    df_credentials, on="credential_id", how="left"
)

In [14]:
df_rules_expanded.shape

(164536, 46)

In [15]:
df_regrouped = (
    df_rules_expanded.groupby("id")
    .agg(
        credential_count=("credential_id", "count"),
        enabled=("enabled", lambda x: (x == True).sum()),
        disabled=("enabled", lambda x: (x == False).sum()),
    )
    .reset_index()
)

df_regrouped["disabled_percentage"] = (df_regrouped["disabled"] * 100.0) / df_regrouped[
    "credential_count"
]
df_regrouped = df_regrouped[df_regrouped["disabled_percentage"] == 100]

In [16]:
df_rules_inactive_creds = df_rules_filtered.merge(df_regrouped, on="id", how="right")

In [17]:
df_rules_inactive_creds.shape

(5, 48)

In [18]:
rule_ids = list(df_regrouped["id"])
rule_ids

['1580', '198662', '575756', '647824', '659340']

In [21]:
# Uncomment to delete
# deleted_rules = api.delete_rules(rule_ids)

Rules deleted successfully.


In [19]:
# df_rules_inactive_creds.to_csv(
#     f"deleted_rules_{datetime.today().strftime('%Y%m%d_%H%M%S')}_with_inactive_creds.csv",
#     index=False,
# )